In [404]:
from collections import defaultdict
from PIL import Image
import random
from tqdm import trange

In [405]:
height = 2400
width = 2400

In [406]:
class Circle():
    def __init__(self, x, y, r, d):
        self.x = x
        self.y = y
        self.r = r
        self.d = d
        self.color = get_trans_colors()
    def draw(self, canvas, depth_map):
        for i in range(-self.r, self.r + 1):  # Adjusted to cover all quadrants.
            for j in range(-self.r, self.r + 1):
                if i**2 + j**2 <= self.r**2:
                    x = self.x + i
                    y = self.y + j
                    if 0 <= x < width and 0 <= y < height:
                        #one pix smaller
                        if i**2 + j**2 <= (self.r - 1)**2:
                            for x_r in range(-1, 1):
                                for y_r in range(-1,1):
                                    depth_map.putpixel((x+x_r, y+y_r), self.d)
                        canvas.putpixel((x, y), self.color)


In [407]:
def random_position():
    import random
    return (random.randint(0, width), random.randint(0, height))

In [408]:
def random_color():
    import random
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

In [409]:
def get_trans_colors():
    colors = [
        (91,206,250),
        (245,169,184),
        (255,255,255)
    ]
    return random.choice(colors)


In [410]:
def doesnt_overlap(pos,circle_radius):
    for layer in boxes:
        for b in boxes[layer]:
            #circle
            if isinstance(b, Circle):
                # if pos in circle 
                if (pos[0] - b.x)**2 + (pos[1] - b.y)**2 <= (circle_radius+50)**2:
                

                    return False
        
    return True

In [416]:
depth_map = Image.new('L', (width, height))
canvas = Image.new('RGB', (width, height))

min_box_width = 4
min_box_height = 4

max_box_width = 45
max_box_height =45

box_layer_count = 10
box_layers=60

boxes = defaultdict(list)
for layer_id in range(box_layers,0,-1):
    for box_id in trange(box_layer_count):
        #scaling based on layer
        circle_radius = int(10 + (layer_id / box_layers) * 40)
        layer_depth = int(255 * (layer_id / box_layers))

        max_tries = 2
        while True:
            max_tries -= 1
            if max_tries == 0:
                break
            position = random_position()
            if doesnt_overlap(position,circle_radius):
                boxes[layer_id].append(Circle(*position, circle_radius, layer_depth))
                break
        
for layer_id, layer in boxes.items():
    print(layer_id, len(layer))
    for b in layer:
        b.draw(canvas, depth_map)

canvas.save('canvas.jpg')
depth_map.save('canvas_depth.jpg')

100%|██████████| 10/10 [00:00<00:00, 11397.57it/s]


100%|██████████| 10/10 [00:00<00:00, 2255.12it/s]


60 10
59 7
58 9
57 10
56 9
55 9
54 6
53 4
52 5
51 6
50 6
49 7
48 6
47 7
46 5
45 8
44 5
43 6
42 4
41 7
40 7
39 8
38 4
37 8
36 3
35 6
34 3
33 6
32 4
31 5
30 5
29 5
28 2
27 5
26 7
25 6
24 4
23 6
22 3
21 3
20 5
19 6
18 3
17 4
16 4
15 6
14 6
13 3
12 5
11 5
10 5
9 7
8 6
7 4
6 5
5 6
4 5
3 8
2 4
1 3


In [417]:

#fill center oval with black
center_x = width // 2
center_y = height // 2
center_r = 600
center_d = 150
center = Circle(center_x, center_y, center_r, center_d)
center.color = (0,0,0)
center.draw(canvas, depth_map)


In [418]:

text = [
    "Trans Rights",
    "Are Human Rights",
]
text_depth = 155
#render text in side 27 font, centered horizontally and vertically on the canvas and depth map

font = ImageFont.truetype("/usr/share/fonts/truetype/ubuntu/UbuntuMono-B.ttf", 121)
font_big = ImageFont.truetype("/usr/share/fonts/truetype/ubuntu/UbuntuMono-B.ttf", 122)
draw = ImageDraw.Draw(canvas)
depth_draw = ImageDraw.Draw(depth_map)

second_line_offset = 120
text_height = 120
for i, line in enumerate(text):
    text_width = draw.textlength(line, font)
    draw.text((int((width - text_width) // 2), int((height - text_height ) // 2 + i * second_line_offset)), line, font=font, fill=(255,00,0))
    depth_draw.text((int((width - text_width) // 2), int((height - text_height) // 2 + i * second_line_offset)), line, font=font, fill=text_depth)

#write cavas.jpg and canvas_depth.jpg
canvas.save('canvas.jpg')
depth_map.save('canvas_depth.jpg')